In [1]:
# !pip install --upgrade 

In [5]:
import os 
import gc

import numpy as np 
import pandas as pd 
import random

from keras.utils import to_categorical

# Suppress warnings in cells to improve readability
import warnings  
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

from google.colab import drive

In [7]:
drive.mount('/content/gdrive')
INPUT_DIR = '/content/gdrive/My Drive/'

Mounted at /content/gdrive


In [11]:
# Upload Quotes.csv
# from google.colab import files
# uploaded = files.upload()

Saving Quotes.csv to Quotes.csv


In [32]:
train = pd.read_csv(INPUT_DIR + "Quotes.csv", delimiter=';', header=None, names=['quote', 'author', 'label'])
feat_cols = "quote"
train = train.dropna()

In [33]:
train.shape

(75961, 3)

In [34]:
all_labels = train.label.unique()
keep = ['amazing','art', 'attitude', 'beauty', 'best','business','change','communication','cool', 'courage','design','dreams', 'education','equality', 'experience', 'failure', 'family', 'famous', 'fear', 'finance', 'fitness', 'forgiveness', 'freedom', 'friendship', 'future',
       'good','great','happiness', 'health','hope','imagination', 'inspirational', 'intelligence','knowledge', 'leadership', 'learning','life', 'love',
       'money','motivational','movingon','patience','positive', 'power', 'relationship','respect', 'smile', 'society','strength', 'success', 'sympathy',
       'thankful', 'time', 'trust','truth','wisdom','work']
drop = [i for i in all_labels if i not in keep]

for item in drop:
    train = train.drop(train.index[train.label == item])

group1 = ['art', 'design', 'education', 'learning', 'knowledge', 'intelligence']
group2 = ['attitude', 'best', 'amazing', 'cool', 'courage',   'good', 'great', 'happiness','hope',  'inspirational','motivational',  'patience', 'positive', 'power','respect', 'smile']
group3 = ['beauty', 'family', 'fitness', 'friendship', 'health', 'love','movingon', 'life','relationship', 'society', 'strength']
group4 = ['work', 'business', 'communication', 'finance', 'leadership', 'money', 'famous', 'success']
group5 = ['change', 'equality', 'fear', 'dreams', 'experience', 'failure', 'forgiveness', 'freedom','future', 'imagination',  'sympathy', 'thankful', 'time', 'trust', 'truth','wisdom']

# create a list of our conditions
conditions = [
    (train['label'].isin(group1)),
    (train['label'].isin(group2)),
    (train['label'].isin(group3)),
    (train['label'].isin(group4)),
    (train['label'].isin(group5))
    ]

# # create a list of the values we want to assign for each condition
label_cols = ['thoughtful', 'emotional', 'personal', 'work', 'aspirations']

# create a new column and use np.select to assign values to it using our lists as arguments
train['label2'] = np.select(conditions, label_cols)

# display updated DataFrame
train.head()

,quote,author,label,label2
1997,Sometimes I think my husband is so amazing tha...,Angelina Jolie,amazing,emotional
1998,If you tell people where to go but not how to ...,George S. Patton,amazing,emotional
1999,It is amazing what you can accomplish if you d...,Harry S. Truman,amazing,emotional
2000,If you set goals and go after them with all th...,Les Brown,amazing,emotional
2001,Nobody in life gets exactly what they thought ...,Conan O'Brien,amazing,emotional


In [35]:
len(all_labels)

117

In [36]:
train.shape

(42030, 4)

In [37]:
train.drop(columns="label", inplace=True)
train.drop(columns="author", inplace=True)
train = train.rename(columns={"label2": "label"})

In [38]:
train.to_csv (f'{INPUT_DIR}QuotesClean.csv', index = False, header=True)

In [39]:
# train = pd.read_csv(INPUT_DIR + "Quotes.csv", delimiter=',', header=0)

In [40]:
train.head()

,quote,label
1997,Sometimes I think my husband is so amazing tha...,emotional
1998,If you tell people where to go but not how to ...,emotional
1999,It is amazing what you can accomplish if you d...,emotional
2000,If you set goals and go after them with all th...,emotional
2001,Nobody in life gets exactly what they thought ...,emotional


In [41]:
train.shape

(42030, 2)

In [58]:
per_cat_count = train.groupby('label').count().reset_index()

In [59]:
per_cat_count.mean()

quote    8406.0
dtype: float64

In [60]:
for i, j in per_cat_count.iterrows(): 
    print(f"{j['label']} | {j['quote']}")  

aspirations | 11478
emotional | 11406
personal | 8379
thoughtful | 5049
work | 5718


In [61]:
counts = [j['quote'] for i,j in per_cat_count.iterrows()]

In [62]:
counts

[11478, 11406, 8379, 5049, 5718]

In [63]:
min_count = float(min(counts))

In [64]:
def getfactors(df):
  fac_list = []
  for i, j in df.iterrows(): 
    val = (j['label'], min_count/float(j['quote']))
    fac_list.append(val)
  return fac_list

In [65]:
factors = getfactors(per_cat_count)

In [66]:
factors

[('aspirations', 0.4398849973863042),
 ('emotional', 0.4426617569700158),
 ('personal', 0.602577873254565),
 ('thoughtful', 1.0),
 ('work', 0.8830010493179433)]

In [67]:
def sampling_k_elements(group, type=2, k=8406):
    if type == 2:
      if len(group) < k:
        return group
      return group.sample(k)
    else:
      if len(group) > k:
          return group
      return group.sample(k, replace=True)

balanced = train.groupby('label').apply(sampling_k_elements).reset_index(drop=True)

In [68]:
balanced_count = balanced.groupby('label').count().reset_index()

In [69]:
balanced_count

,label,quote
0,aspirations,8406
1,emotional,8406
2,personal,8379
3,thoughtful,5049
4,work,5718


In [70]:
balanced.to_csv (f'{INPUT_DIR}BalancedQuotesClean.csv', index = False, header=True)

In [71]:
new = balanced.sample(5)
new
tes = []
for item in new['quote']:
  tes.append(item)
tes

['On what rests the hope of the republic? One country one language one flag!',
 'We call that person who has lost his father an orphan and a widower that man who has lost his wife. But that man who has known the immense unhappiness of losing a friend by what name do we call him? Here every language is silent and holds its peace in impotence.',
 'Our universities and museums are respected around the country.',
 'Magnanimity in politics is not seldom the truest wisdom and a great empire and little minds go ill together.',
 'Our goals can only be reached through a vehicle of a plan in which we must fervently believe and upon which we must vigorously act. There is no other route to success.']

In [72]:
l=['thoughtful', 'emotional', 'personal', 'work', 'aspirations']

# Get the numerical ids of column label
balanced['label']=balanced.label.astype('category')

Y = balanced.label.cat.codes
balanced['label']=Y

# Print initial shape
print(Y.shape)

# One-hot encode the indexes
Y = to_categorical(Y)

# Check the new shape of the variable
print(Y.shape)

(35958,)
(35958, 5)
